In [1]:
import re
import operator
import numpy as np
import seaborn as sns
import pandas as pd
import pickle as pickle #data processing
from collections import Counter #tokenization
import keras #ML
#import processing as pr #helper
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
shakespeare_db = pd.read_csv("Shakespeare_data.csv")

In [3]:
shakespeare_db

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil
8,9,Henry IV,1.0,1.1.6,KING HENRY IV,Shall daub her lips with her own children's bl...
9,10,Henry IV,1.0,1.1.7,KING HENRY IV,"Nor more shall trenching war channel her fields,"


In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(shakespeare_db['PlayerLine'])

In [5]:
uniques = vectorizer.get_feature_names()

In [6]:
word_array = X.toarray()

In [7]:
def keywords(docs, k):
    tok = Tokenizer()
    tok.fit_on_texts(docs)
    tfidf_matrix = tok.texts_to_matrix(docs,mode='tfidf')
    tfidf_scores = np.sum(tfidf_matrix, axis=0)
    return sorted(dict(zip(tok.word_index,tfidf_scores)).items(),key=lambda x:x[1])

In [8]:
docs = ["The sky is blue", "I love the sky", "let's kill some seagulls"]

In [9]:
keywords(docs,5)

[('the', 0.0),
 ('blue', 0.9162907318741551),
 ('i', 0.9162907318741551),
 ('love', 0.9162907318741551),
 ("let's", 0.9162907318741551),
 ('kill', 0.9162907318741551),
 ('some', 0.9162907318741551),
 ('seagulls', 0.9162907318741551),
 ('sky', 1.3862943611198906),
 ('is', 1.3862943611198906)]

In [10]:
def alt_keywords(docs, k):
    lines = [line.rstrip() for line in open('SmartStoplist.txt')]
    stop_list = set(lines)
    vectorizer = TfidfVectorizer(stop_words=stop_list)
    tfidf_matrix = vectorizer.fit_transform(docs)
    tfidf_scores = np.sum(tfidf_matrix,axis=0)
    tfidf_scores = np.ravel(tfidf_scores)
    return sorted(dict(zip(vectorizer.get_feature_names(),tfidf_scores)).items(),key=lambda x:x[1], reverse=True)[:k]

In [11]:
alt_keywords(docs,5)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'll', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('sky', 1.2106970162125832),
 ('blue', 0.7959605415681652),
 ('love', 0.7959605415681652),
 ('kill', 0.7071067811865476),
 ('seagulls', 0.7071067811865476)]

In [12]:
def get_vocab(lst): #tokenizing
    vocabcount, vocab = Counter(w for txt in lst for w in txt.split())
    return vocab, vocabcount


In [13]:
vocab, vocabcount = get_vocab(docs)

ValueError: too many values to unpack (expected 2)

In [14]:
path = 'glove.6b.zip'
glove_weights = get_glove_weights(path, origin="https://nlp.stanford.edu/data/glove.6b.zip")


NameError: name 'get_glove_weights' is not defined

In [15]:
FN = 'vocabulary-embedding'

In [16]:

seed=42


In [17]:
vocab_size = 40000

In [18]:

embedding_dim = 100


In [19]:

lower = False # dont lower case the text

In [20]:

import pickle as pickle
FN0 = 'tokens' # this is the name of the data file which I assume you already have
with open('data/%s.pkl'%FN0, 'rb') as fp:
    heads, desc, keywords = pickle.load(fp) # keywords are not used in this project

FileNotFoundError: [Errno 2] No such file or directory: 'data/tokens.pkl'

In [21]:
def histogram(words):
    #words_list = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", words)
    words_list = re.split("\W*[^\'\w+\']", words)
    #thanks to Martijn Pieters for the above split
    word_dictionary = {}
    counter = len(words_list)
    i = 0
    while i < counter:
        word_dictionary[words_list[i]] = 1
        n = i+1
        while n < counter:
            if words_list[i] == words_list[n]:
                word_dictionary[words_list[i]] += 1
                words_list.pop(n)
                counter -= 1
                n -= 1
            n += 1
        i += 1
    return word_dictionary
def unique_words(histogram):
    return len(histogram)
def frequency(word, histogram):
    return histogram[word]
def create_sorted_array(histogram):
    new_array = []
    count = 0
    for i in histogram:
        new_array.append([histogram[i]])
        new_array[count].append(i)
        count += 1
    new_array.sort()
    return new_array

In [22]:
hamlet = shakespeare_db[shakespeare_db['Play']=='Hamlet']

In [23]:
hamlet = hamlet['PlayerLine']

In [24]:
def play_string(play):
    string = ""
    for i in play:
        string += i
        string += " "
    return string

In [25]:
hamlet_string = play_string(hamlet)

In [26]:
hamlet_histogram = histogram(hamlet_string)

In [27]:
hamlet_list = create_sorted_array(hamlet_histogram)

In [28]:
hamlet_list[len(hamlet_list)-600]

[6, 'toward']

In [31]:
alt_keywords(hamlet,10)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'll', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('lord', 118.25534183931137),
 ('hamlet', 48.721812430553356),
 ('good', 47.502744404792715),
 ('king', 40.60075069635388),
 ('sir', 37.606794647308455),
 ('thou', 36.278785770811545),
 ('enter', 35.61883488074113),
 ('speak', 34.99068510128831),
 ('tis', 33.040598859697745),
 ('exit', 32.12167468836296)]